# DATA
## Description

Как первое приближение, собираем данные для premium и professional лиг dota 2

Раньше соревнования делились на 4 типа:
1. Premier -- международные соревнования, с типичным призовым фондом от 300,000 USD
2. Major -- от 50,000 USD -- лучшие команды всё ещё принимают участие
3. Minor -- от 3000 USD -- менее престижные, но с большой конкуренцией
4. Остальные

Но Valve отказался от этой системы в не так давно, поэтому соберём данные для примерное первых трех тиров, которые разделим на premium и professional. Так в выборку не попадут любители, а все основные турниры будут покрыты.

Для двух профессиональных лиг соберём все матчи за последний год -- между the international 2021 и the international 2020 -- главными турнирами в dota 2, которыми заканчивается любой сезон.

Для премиум лиги получим приблизительно 2000 игр между командами, однако в рамках встречи на одном турнире две команды могут сыграть несколько игр друг с другом (от 3 до 5), поэтому количество уникальных встреч будет в несколько раз меньше

для профессиональной лиги соберём примерно 5000 игр за тот же промежуток

Так же для каждой встречи соберём состав игроков и героев, за которых они играли ― в каждой команде по 5 игроков, в матче принимают участие 2 команды. Состав команды относительно стабилен в течение сезона, однако игроки свободно могут переходить между командами в трансферное окно, которое открыто весь год, за исключением месяца, в который проходит the International

В каждом матче каждый игрок выбирает героя, которым управляет в течении матча. Игроки могут лучше играть на определенных героях, однако почти каждый матч композиция меняется, так как обе команды выбирают героев поочерёдно.


финальный вид датасета

| Турнир            | дата матча | команда 1 | команда 2   | победитель                | составы                     | герои              |
|-------------------|------------|-----------|-------------|---------------------------|-----------------------------|--------------------|
| The International | 2021-10-17 |  PSG.LGD  | Team Spirit | Team Spirit               | [ Yatoro,  TORONTOTOKYO...] | [Magnus, Bane ...] |
| DPC Winter 21 League (EU) presented by DreamLeague         | 2021-01-18 | Alliance | Liquid      | Alliance |  [MATUMBAMAN, miCKe, ...]   | [Lion, Tidehunter,  ...]    |


Структура данных:

dota_premium_minimal.parquet -- матчи premium лига

dota_professional_minimal.parquet -- мачти professional лига

datdota_Leagues.csv" -- названия турниров, объединяем с матчами по Series

dota_macthes -- имена спортсменов и выбранные герои, объединяем с матчами по Match ID. Так же внутри много дополнительных характеристик, например убийства, смерти и опыт

## Data collection

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import time

## premium

In [4]:
URL = "https://www.datdota.com/matches?tier=premium"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [15]:
data = []
table = soup.find('table')
table_body = table.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

In [30]:
df_premium = pd.DataFrame(data, columns=['Match ID', 'Series', 'League',
                                         'Start Date/Time', 'Match Length',
    'Duration (secs)', 'Kills Score', 'Unnamed: 7', 'Winner',
    'Radiant Team', 'Radiant Players', 'Dire Team', 'Dire Players'])
df_premium

,Match ID,Series,League,Start Date/Time,Match Length,Duration (secs),Kills Score,Unnamed: 7,Winner,Radiant Team,Radiant Players,Dire Team,Dire Players
0,6293408192,619517,13725,26 Nov 2021 07:30,51:19,3079,32-30,"32,30",Radiant,cybercats,"Airu, Petru44o, USHUSHUSH, Fezor, Shigetsu",V-Gaming,"Naive-, ArrOw, destroyed, antoha, HAze"
1,6293396000,619513,13725,26 Nov 2021 07:22,50:51,3051,32-32,"32,32",Dire,HYDRA,"Zayac, Askold, Blizzy, Dream`, Le don",label bmd,"myart, discay, fam0us, Nicky`Cool , ri"
2,6293297950,619485,13725,26 Nov 2021 06:15,46:52,2812,14-36,"14,36",Dire,Miaww^^,"Kidaro, nattynarwhal_, Antares, Ga1k, DarkLord^",V-Gaming,"Naive-, ArrOw, destroyed, antoha, HAze"
3,6293285430,619481,13725,26 Nov 2021 06:07,40:59,2459,32-34,"32,34",Dire,YNGBLD,"Otaker, xxx, Volshebnik, TheSpeedy, Fill",label bmd,"myart, discay, fam0us, Nicky`Cool , ri"
4,6293248016,619467,13725,26 Nov 2021 05:43,39:09,2349,34-16,"34,16",Radiant,cybercats,"Airu, Petru44o, USHUSHUSH, Fezor, Shigetsu",Legion Gaming,"baxadoto, kur0patbI4, Aliback, HardL, Overkill~"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4779722555,331595,10979,24 May 2019 03:48,29:54,1794,25-13,"25,13",Radiant,Ninjas in Pyjamas,"33, Ppd, Fata, Ace, Saksa",Alliance,"INSaNiA, Boxi, Qojqva, MiCKe, Taiga"
4996,4779424867,331559,10979,24 May 2019 02:03,52:26,3146,29-33,"29,33",Radiant,Ninjas in Pyjamas,"33, Ppd, Fata, Ace, Saksa",The Final Tribe,"Handsken, Xibbe, Era, Frost, Chessie"
4997,4779407768,331560,10979,24 May 2019 01:56,42:48,2568,55-29,"55,29",Radiant,Alliance,"INSaNiA, Boxi, Qojqva, MiCKe, Taiga",Aachen City Esports,"LeBronDota, DM, Ecnart, Funn1k, V-Tune"
4998,4779285288,331560,10979,24 May 2019 01:00,30:38,1838,7-29,"7,29",Dire,Aachen City Esports,"LeBronDota, DM, Ecnart, Funn1k, V-Tune",Alliance,"INSaNiA, Boxi, Qojqva, MiCKe, Taiga"


In [31]:
df_premium.to_parquet('dota_premium_minimal.parquet')

### professional

In [39]:
URL = "https://www.datdota.com/matches?tier=professional"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [40]:
data = []
table = soup.find('table')
table_body = table.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

In [41]:
df_professional = pd.DataFrame(data, columns=['Match ID', 'Series', 'League',
                                         'Start Date/Time', 'Match Length',
                                         'Duration (secs)', 'Kills Score', 'Unnamed: 7', 'Winner',
                                         'Radiant Team', 'Radiant Players', 'Dire Team', 'Dire Players'])
df_professional

,Match ID,Series,League,Start Date/Time,Match Length,Duration (secs),Kills Score,Unnamed: 7,Winner,Radiant Team,Radiant Players,Dire Team,Dire Players
0,6292119239,619084,13379,25 Nov 2021 10:19,33:50,2030,10-26,"10,26",Dire,Entity,"Stormstormer, Fishman, Crystallis, Tobi, suzuya",HellRaisers,"Malady.LFT, Время в никуда, watson;,.', Gilgir..."
1,6292041530,619084,13379,25 Nov 2021 09:02,38:32,2312,46-25,"46,25",Radiant,Entity,"Stormstormer, Fishman, Crystallis, Tobi, suzuya",HellRaisers,"Malady.LFT, Время в никуда, watson;,.', Gilgir..."
2,6291972971,619084,13379,25 Nov 2021 08:04,29:09,1749,27-16,"27,16",Radiant,HellRaisers,"Malady.LFT, Время в никуда, watson;,.', Gilgir...",Entity,"Stormstormer, Fishman, Crystallis, Tobi, suzuya"
3,6291891317,619084,13379,25 Nov 2021 07:01,38:01,2281,27-24,"27,24",Radiant,Entity,"Stormstormer, Fishman, Crystallis, Tobi, suzuya",HellRaisers,"Malady.LFT, Время в никуда, watson;,.', Gilgir..."
4,6291811058,619084,13379,25 Nov 2021 06:00,31:33,1893,20-13,"20,13",Radiant,HellRaisers,"Malady.LFT, Время в никуда, watson;,.', Gilgir...",Entity,"Stormstormer, Fishman, Crystallis, Tobi, suzuya"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,5583982350,474768,12229,25 Aug 2020 10:13,49:03,2943,20-26,"20,26",Dire,Aster.Aries,"Rico, harutya, Lww, ShengSi, phoenix",Geek Fam,"Karl, Xepher, Whitemon, Abed, Kuku"
4996,5583922861,474750,12229,25 Aug 2020 09:04,28:50,1730,24-12,"24,12",Radiant,T1,"Poloson, Meracle, Jhocam, InYourdreaM, Sam H",QuestionMark,"LionaX, MyPro, Kanom``Pang, Leaf, MikaeL~"
4997,5583863565,474750,12229,25 Aug 2020 08:00,28:07,1687,24-11,"24,11",Radiant,T1,"Poloson, Meracle, Jhocam, InYourdreaM, Sam H",QuestionMark,"LionaX, MyPro, Kanom``Pang, Leaf, MikaeL~"
4998,5583629438,474697,12245,25 Aug 2020 02:28,47:33,2853,29-32,"29,32",Radiant,Boonz + Goonz,"Monster, Lordboonz, RRL, Froogoss, Yuma",PACE,"talianoG, Aikster, Lies, Stars, Deano"


In [42]:
df_professional.to_parquet("dota_professioanl_minimal.parquet")

### leagues

In [ ]:
## https://www.datdota.com/leagues
df_leagues = pd.read_csv('datdota_Leagues.csv')

## heroes

In [63]:
df_premium  = pd.read_parquet('dota_premium_minimal.parquet')
df_premium

,Match ID,Series,League,Start Date/Time,Match Length,Duration (secs),Kills Score,Unnamed: 7,Winner,Radiant Team,Radiant Players,Dire Team,Dire Players
0,6293408192,619517,13725,26 Nov 2021 07:30,51:19,3079,32-30,"32,30",Radiant,cybercats,"Airu, Petru44o, USHUSHUSH, Fezor, Shigetsu",V-Gaming,"Naive-, ArrOw, destroyed, antoha, HAze"
1,6293396000,619513,13725,26 Nov 2021 07:22,50:51,3051,32-32,"32,32",Dire,HYDRA,"Zayac, Askold, Blizzy, Dream`, Le don",label bmd,"myart, discay, fam0us, Nicky`Cool , ri"
2,6293297950,619485,13725,26 Nov 2021 06:15,46:52,2812,14-36,"14,36",Dire,Miaww^^,"Kidaro, nattynarwhal_, Antares, Ga1k, DarkLord^",V-Gaming,"Naive-, ArrOw, destroyed, antoha, HAze"
3,6293285430,619481,13725,26 Nov 2021 06:07,40:59,2459,32-34,"32,34",Dire,YNGBLD,"Otaker, xxx, Volshebnik, TheSpeedy, Fill",label bmd,"myart, discay, fam0us, Nicky`Cool , ri"
4,6293248016,619467,13725,26 Nov 2021 05:43,39:09,2349,34-16,"34,16",Radiant,cybercats,"Airu, Petru44o, USHUSHUSH, Fezor, Shigetsu",Legion Gaming,"baxadoto, kur0patbI4, Aliback, HardL, Overkill~"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4779722555,331595,10979,24 May 2019 03:48,29:54,1794,25-13,"25,13",Radiant,Ninjas in Pyjamas,"33, Ppd, Fata, Ace, Saksa",Alliance,"INSaNiA, Boxi, Qojqva, MiCKe, Taiga"
4996,4779424867,331559,10979,24 May 2019 02:03,52:26,3146,29-33,"29,33",Radiant,Ninjas in Pyjamas,"33, Ppd, Fata, Ace, Saksa",The Final Tribe,"Handsken, Xibbe, Era, Frost, Chessie"
4997,4779407768,331560,10979,24 May 2019 01:56,42:48,2568,55-29,"55,29",Radiant,Alliance,"INSaNiA, Boxi, Qojqva, MiCKe, Taiga",Aachen City Esports,"LeBronDota, DM, Ecnart, Funn1k, V-Tune"
4998,4779285288,331560,10979,24 May 2019 01:00,30:38,1838,7-29,"7,29",Dire,Aachen City Esports,"LeBronDota, DM, Ecnart, Funn1k, V-Tune",Alliance,"INSaNiA, Boxi, Qojqva, MiCKe, Taiga"


In [81]:
dota_premium_heroes_collected = pd.read_parquet("dota_premium_heroes.parquet")
dota_premium_heroes_collected

,Dire Heroes,Dire Players,Match ID,Radian Players,Radiant Heroes
0,"[ursa, disruptor, queenofpain, phoenix, rattle...","[V-Tune, ALWAYSWANNAFLY, Iceberg, RodjER, Gene...",5857415725,"[Nightfall, Gpk, DM, Save-, Kingslayer]","[monkey_king, deathprophet, shredder, lion, or..."
1,"[weaver, shadowdemon, undying, lina, slardar]","[Naive-, ArrOw, antoha, HAze, destroyed]",6293408192,"[Shigetsu, Petru44o, Fezor, Airu, USHUSHUSH]","[facelessvoid, snapfire, jakiro, tidehunter, b..."
2,"[phantomlancer, disruptor, monkey_king, grimst...","[ri, fam0us, myart, Nicky`Cool , discay]",6293396000,"[Zayac, Blizzy, Le don, Dream`, Askold]","[undying, dragonknight, weaver, terrorblade, i..."
3,"[ursa, wisp, weaver, lina, pangolier]","[Naive-, antoha, ArrOw, HAze, destroyed]",6293297950,"[Antares, Kidaro, Ga1k, nattynarwhal_, DarkLord^]","[techies, disruptor, tidehunter, windrunner, p..."
4,"[weaver, shredder, darkwillow, magnataur, medusa]","[fam0us, Nicky`Cool , myart, discay, ri]",6293285430,"[Otaker, Volshebnik, xxx, TheSpeedy, Fill]","[dawnbreaker, morphling, spiritbreaker, bane, ..."
...,...,...,...,...,...
1340,"[juggernaut, voidspirit, shredder, lion, witch...","[Nightfall, Gpk, DM, Save-, Kingslayer]",5961010983,"[Dukalis, Desperate-, Krylat, MeLes, Astral]","[grimstroke, mars, skeletonking, batrider, tusk]"
1341,"[bloodseeker, mars, earthspirit, beastmaster, ...","[Mingatte, Lissex, Zxca, J e i m a r i, Thugger]",5961000021,"[Rises, Alone, Drakeel, Tano, Cara]","[trollwarlord, leshrac, venomancer, nyxassassi..."
1342,"[necrolyte, skeletonking, tusk, ancientapparit...","[Chshrct, DyrachYO, Yamich, pantomem, Re1bl]",5960902003,"[CemaTheSlayeR, Ghostik, Fn, Bignum, Palantimos]","[grimstroke, abyssalunderlord, voidspirit, ear..."
1343,"[lina, doombringer, juggernaut, grimstroke, ma...","[Bignum, Fn, Palantimos, CemaTheSlayeR, Ghostik]",5960787505,"[DyrachYO, Re1bl, Chshrct, Yamich, pantomem]","[gyrocopter, puck, slardar, phoenix, tusk]"


In [ ]:
for match_id in df_premium['Match ID'].tolist():
    if match_id not in dota_premium_heroes_collected['Match ID'].tolist():
        url = "https://www.datdota.com/api/matches/" + match_id
        response = requests.get(url)

        json_dict = response.json()

        radiant_players = []
        radiant_heroes = []

        dire_players = []
        dire_heroes = []

        match_id = json_dict['data']['match_id']

        for player in json_dict['data']['radiant']['player_performances']:
                radiant_players.append(player['player']['nickname'])
                radiant_heroes.append(player['performance']['hero']['short_name'])

        for player in json_dict['data']['dire']['player_performances']:
            dire_players.append(player['player']['nickname'])
            dire_heroes.append(player['performance']['hero']['short_name'])

        d = {"match": {'Match ID': match_id,
                       'Radian Players': radiant_players, 'Radiant Heroes': radiant_heroes,
                       'Dire Players' : dire_players, 'Dire Heroes' : dire_heroes}
             }
        match_row = pd.DataFrame(data=d).transpose().reset_index(drop=True)
        dota_premium_heroes = pd.read_parquet("dota_premium_heroes.parquet")
        dota_premium_heroes_2 = pd.concat([dota_premium_heroes, match_row], ignore_index=True)
        dota_premium_heroes_2.to_parquet("dota_premium_heroes.parquet")
        time.sleep(2)

In [72]:
dota_premium_heroes_collected = pd.read_parquet("dota_premium_heroes.parquet")

,Dire Heroes,Dire Players,Match ID,Radian Players,Radiant Heroes
0,"[ursa, disruptor, queenofpain, phoenix, rattle...","[V-Tune, ALWAYSWANNAFLY, Iceberg, RodjER, Gene...",5857415725,"[Nightfall, Gpk, DM, Save-, Kingslayer]","[monkey_king, deathprophet, shredder, lion, or..."
1,"[weaver, shadowdemon, undying, lina, slardar]","[Naive-, ArrOw, antoha, HAze, destroyed]",6293408192,"[Shigetsu, Petru44o, Fezor, Airu, USHUSHUSH]","[facelessvoid, snapfire, jakiro, tidehunter, b..."
2,"[phantomlancer, disruptor, monkey_king, grimst...","[ri, fam0us, myart, Nicky`Cool , discay]",6293396000,"[Zayac, Blizzy, Le don, Dream`, Askold]","[undying, dragonknight, weaver, terrorblade, i..."
3,"[ursa, wisp, weaver, lina, pangolier]","[Naive-, antoha, ArrOw, HAze, destroyed]",6293297950,"[Antares, Kidaro, Ga1k, nattynarwhal_, DarkLord^]","[techies, disruptor, tidehunter, windrunner, p..."
4,"[weaver, shredder, darkwillow, magnataur, medusa]","[fam0us, Nicky`Cool , myart, discay, ri]",6293285430,"[Otaker, Volshebnik, xxx, TheSpeedy, Fill]","[dawnbreaker, morphling, spiritbreaker, bane, ..."
...,...,...,...,...,...
1340,"[juggernaut, voidspirit, shredder, lion, witch...","[Nightfall, Gpk, DM, Save-, Kingslayer]",5961010983,"[Dukalis, Desperate-, Krylat, MeLes, Astral]","[grimstroke, mars, skeletonking, batrider, tusk]"
1341,"[bloodseeker, mars, earthspirit, beastmaster, ...","[Mingatte, Lissex, Zxca, J e i m a r i, Thugger]",5961000021,"[Rises, Alone, Drakeel, Tano, Cara]","[trollwarlord, leshrac, venomancer, nyxassassi..."
1342,"[necrolyte, skeletonking, tusk, ancientapparit...","[Chshrct, DyrachYO, Yamich, pantomem, Re1bl]",5960902003,"[CemaTheSlayeR, Ghostik, Fn, Bignum, Palantimos]","[grimstroke, abyssalunderlord, voidspirit, ear..."
1343,"[lina, doombringer, juggernaut, grimstroke, ma...","[Bignum, Fn, Palantimos, CemaTheSlayeR, Ghostik]",5960787505,"[DyrachYO, Re1bl, Chshrct, Yamich, pantomem]","[gyrocopter, puck, slardar, phoenix, tusk]"


In [73]:
df_professional = pd.read_parquet('dota_professional_heroes.parquet')
df_professional

,Dire Heroes,Dire Players,Match ID,Radian Players,Radiant Heroes
0,"[ursa, disruptor, queenofpain, phoenix, rattle...","[V-Tune, ALWAYSWANNAFLY, Iceberg, RodjER, Gene...",5857415725,"[Nightfall, Gpk, DM, Save-, Kingslayer]","[monkey_king, deathprophet, shredder, lion, or..."


In [ ]:
for match_id in df_professional['Match ID'].tolist():
    url = "https://www.datdota.com/api/matches/" + match_id
    response = requests.get(url)
    json_dict = response.json()

    radiant_players = []
    radiant_heroes = []

    dire_players = []
    dire_heroes = []

    match_id = json_dict['data']['match_id']

    for player in json_dict['data']['radiant']['player_performances']:
        radiant_players.append(player['player']['nickname'])
        radiant_heroes.append(player['performance']['hero']['short_name'])

    for player in json_dict['data']['dire']['player_performances']:
        dire_players.append(player['player']['nickname'])
        dire_heroes.append(player['performance']['hero']['short_name'])

    d = {"match": {'Match ID': match_id,
                       'Radian Players': radiant_players, 'Radiant Heroes': radiant_heroes,
                       'Dire Players' : dire_players, 'Dire Heroes' : dire_heroes}
             }
    match_row = pd.DataFrame(data=d).transpose().reset_index(drop=True)
    dota_professional_heroes = pd.read_parquet("dota_professional_heroes.parquet")
    dota_professional_heroes2 = pd.concat([dota_professional_heroes, match_row], ignore_index=True)
    dota_professional_heroes2.to_parquet("dota_professional_heroes.parquet")
    time.sleep(1)

In [ ]:
date_list = [
        '10%2F12%2F2020"&before=20%2F12%2F2020',
        '01%2F12%2F2020"&before=10%2F12%2F2020',
        '20%2F11%2F2020"&before=01%2F12%2F2020',

'10%2F11%2F2020"&before=20%2F11%2F2020',
'01%2F11%2F2020"&before=10%2F11%2F2020',
'20%2F10%2F2020"&before=01%2F11%2F2020',
'10%2F10%2F2020"&before=20%2F10%2F2020',
'01%2F10%2F2020"&before=10%2F10%2F2020',
'20%2F09%2F2020"&before=01%2F10%2F2020',
'10%2F09%2F2020"&before=20%2F09%2F2020',
'01%2F09%2F2020"&before=10%2F09%2F2020',
'20%2F08%2F2020"&before=01%2F09%2F2020',
'10%2F08%2F2020"&before=20%2F08%2F2020',
'01%2F08%2F2020"&before=10%2F08%2F2020',
'20%2F07%2F2020"&before=01%2F08%2F2020',
'10%2F07%2F2020"&before=20%2F07%2F2020',
'01%2F07%2F2020"&before=10%2F07%2F2020',
'20%2F06%2F2020"&before=01%2F07%2F2020',
    ]

In [115]:
for date_list_one in date_list:
    URL = "http://datdota.com/players/single-performance?patch=7.30&patch=7.29&patch=7.28&patch=7.27&patch=7.26&patch=7.25&patch=7.24&patch=7.23&patch=7.22&patch=7.21&patch=7.20&patch=7.19&patch=7.18&patch=7.17&patch=7.16&patch=7.15&patch=7.14&patch=7.13&patch=7.12&patch=7.11&patch=7.10&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&patch=6.87&patch=6.86&patch=6.85&patch=6.84&patch=6.83&patch=6.82&patch=6.81&patch=6.80&patch=6.79&patch=6.78&patch=6.77&patch=6.76&patch=6.75&patch=6.74&after="+date_list_one+"&duration=0%3B200&duration-value-from=0&duration-value-to=200&tier=1&tier=2&valve-event=does-not-matter"
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, "html.parser")

    data = []
    table = soup.find('table')
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values

    match_small = pd.DataFrame(data, columns=["Match ID", "Hero", "Player", "Result", "Kills", "Deaths", "Assists", "KDA"	, "GPM"	, "XPM"	, "Last Hits", "Denies", 	"LVL",	"HD",	"TD", "HH", "GS",	"End Game Items"])

    dota_matches = pd.read_parquet("dota_matches.parquet")
    dota_matches2 = pd.concat([dota_matches, match_small], ignore_index=True)
    dota_matches2.to_parquet("dota_matches.parquet")

    time.sleep(5)

In [116]:
dota_matches = pd.read_parquet("dota_matches.parquet")
dota_matches

,Match ID,Hero,Player,Result,Kills,Deaths,Assists,KDA,GPM,XPM,Last Hits,Denies,LVL,HD,TD,HH,GS,End Game Items
0,6068802013,Treant Protector,Bloody Nine,Loss,2,12,15,1.42,218,370,33,3,19,11344,0,6519,9160,Tranquil Boots\nGlimmer Cape\nHoly Locket\nObs...
1,6068802013,Spectre,EternaLEnVy,Win,7,3,19,8.67,726,848,415,8,28,39394,8196,0,27895,Manta Style\nHood of Defiance\nPower Treads\nE...
2,6068802013,Nyx Assassin,Zfreek,Win,1,8,16,2.12,271,459,40,11,21,11495,0,0,11015,Aghanim's Scepter\nDust of Appearance\nTranqui...
3,6068802013,Viper,Kingrd,Win,11,5,16,5.40,605,690,317,15,26,33891,12563,0,23305,Hurricane Pike\nMagic Wand\nBlack King Bar\nBo...
4,6068802013,Dragon Knight,Ryoya,Win,14,2,12,13.00,699,841,415,14,28,27447,18586,592,30140,Overwhelming Blink\nBlack King Bar\nBloodthorn...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68785,5478856026,Ursa,汤 达 人,Loss,8,5,6,2.80,553,623,310,4,22,17412,266,0,18820,Magic Wand\nPower Treads\nAbyssal Blade\nBlack...
68786,5478856026,Void Spirit,Piao,Loss,8,5,11,3.80,419,610,163,14,21,24758,354,0,12735,Phase Boots\nEul's Scepter of Divinity\nOgre A...
68787,5478856026,Enigma,InJuly,Loss,3,9,12,1.67,409,396,226,24,17,19917,1773,3123,13145,Black King Bar\nNecronomicon 1\nMagic Stick\nG...
68788,5478856026,Rubick,Moriarty,Loss,2,13,15,1.31,234,313,66,7,14,12500,0,0,7130,Tranquil Boots\nMagic Wand\nGlimmer Cape\nAeth...


In [117]:
dota_matches['Match ID'].min()

'5478856026'

In [118]:
dota_matches['Match ID'].max()

'6068802013'

In [121]:
len(dota_matches['Match ID'])/10

6879.0